<a href="https://colab.research.google.com/github/thofes/SJ_distance/blob/main/Resnet_Mark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

Mounted at /content.gdrive


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation, AveragePooling2D, Input, Flatten, add

# Load the data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content.gdrive/MyDrive/Dataset_Resnet/train",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=32
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content.gdrive/MyDrive/Dataset_Resnet/test",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(256, 256),
    batch_size=32
)

# Normalize the data
def normalize(x, y):
    x = tf.image.per_image_standardization(x)
    return x, y

train_ds = train_ds.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle and batch the data
train_ds = train_ds.shuffle(buffer_size=1000).batch(32)
val_ds = val_ds.batch(32)



def resnet_block(inputs, filters, strides=1):
    x = Conv2D(filters, kernel_size=(3,3), strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=(3,3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def ResNet(input_shape=(256,256,3), classes=2):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=(7,7), strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=(3,3), strides=2)(x)
    
    x = resnet_block(x, 64)
    x = resnet_block(x, 64)
    x = resnet_block(x, 64)
    
    x = resnet_block(x, 128, strides=2)
    x = resnet_block(x, 128)
    x = resnet_block(x, 128)
    
    x = resnet_block(x, 256, strides=2)
    x = resnet_block(x, 256)
    x = resnet_block(x, 256)
    
    x = resnet_block(x, 512, strides=2)
    x = resnet_block(x, 512)
    x = resnet_block(x, 512)
    
    x = AveragePooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax')(x)
    
    model = keras.Model(inputs, x, name='resnet')
    return model

model = ResNet()
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def reshape(x, y):
    x = tf.reshape(x, (-1, 256, 256, 3))
    return x, y

train_ds = train_ds.map(reshape, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(reshape, num_parallel_calls=tf.data.AUTOTUNE)



# Train the model
batch_size = 16
model.fit(train_ds, epochs=10, validation_data=val_ds, batch_size=8)


# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

Found 1000 files belonging to 2 classes.
Using 800 files for training.
Found 501 files belonging to 2 classes.
Using 100 files for validation.
Model: "resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_50 (Conv2D)          (None, 128, 128, 64)      9472      
                                                                 
 batch_normalization_50 (Bat  (None, 128, 128, 64)     256       
 chNormalization)                                                
                                                                 
 activation_50 (Activation)  (None, 128, 128, 64)      0         
                                                                 
 average_pooling2d_4 (Averag  (None, 63, 63, 64)       0         
 ePooling2D)                                     

ResourceExhaustedError: ignored

The image_dataset_from_directory function takes in the path to the directory containing the images, the validation split, and the image size you want to use. It automatically labels the images based on the directory structure, so the images in the same directory are considered to be of the same class.

The function returns a tf.data.Dataset object that can be used to train the model. The map function is used to apply normalization to the data. The data is also shuffled and batched to improve performance during training.

You can also use tf.keras.preprocessing.image.ImageDataGenerator class to load and preprocess the images, this class also provides data augmentation techniques which can be useful to improve the performance of the model.

- path/to/train_data
    - in_air
        - image1.jpg
        - image2.jpg
        - ...
    - on_ground
        - image1.jpg
        - image2.jpg
        - ...

oder:
  - path/to/train_data
    - train
        - in_air
            - image1.jpg
            - image2.jpg
            - ...
        - on_ground
            - image1.jpg
            - image2.jpg
            - ...
    - validation
        - in_air
            - image1.jpg
            - image2.jpg
            - ...
        - on_ground
            - image1.jpg
            - image2.jpg
            - ...

  
  You can use any image file format that is supported by TensorFlow, such as JPG or PNG. Make sure that all images have the same size (256x256 in this case).

Finally, if you have images in different file format or size, you can use libraries such as OpenCV to convert them to the appropriate format and size before using the function to load them.

In [ ]:
model.save('resnet.h5')